In [2]:
import MySQLdb as mdb
import datetime
import sys
import os
from peewee import *
import glob
from collections import defaultdict
import json
import shortuuid
import random
from datetime import datetime

import PIL
from PIL import Image
from PIL import ImageFilter
from os import listdir
from os.path import isfile, join
from shutil import copyfile
import numpy

config = json.load(open('config.json', 'r'))

con = mdb.connect(host="localhost", user=config['db_user'], db=config['db_name'], passwd=config['db_pass'])
database = MySQLDatabase(config['db_name'], user=config['db_user'], password=config['db_pass'])
class BaseModel(Model):
    class Meta:
        database=database

class Video(BaseModel):
    youtube_id = CharField(primary_key=True, index=True)
    video_name = CharField()
    label = CharField()
    video_Type = CharField()
    time_start = CharField()
    time_end = CharField()
    split = CharField()
    url = CharField()
    numHitsFinished = CharField()


#class Label(BaseModel):
#    labelId = CharField(primary_key=True, index=True)
#    label = CharField()
#    video = ForeignKeyField(Video)

class AMTHits(BaseModel):
    id = CharField(primary_key=True)
    socketId = CharField()
    assignmentId = CharField()
    workerId = CharField()
    approve = CharField(default='notApprove')
    hitId = CharField()
    status = CharField()
    isPaid = BooleanField(default='0')
    bonus = IntegerField(default=0)
    hitIden = CharField()
    comment = CharField()
    video = ForeignKeyField(Video)
    #label = ForeignKeyField(Label)
    created_at = IntegerField(default=int(datetime.now().strftime('%s')))
    completed_at = IntegerField(default=0)

class Feedback(BaseModel):
    workerId = CharField()
    hitId = CharField()
    assignmentId = CharField()
    sequenceId = CharField()
    feedback = TextField()

class Question(BaseModel):
    id = CharField(primary_key=True, index=True)
    question = CharField()
    youtube_id = ForeignKeyField(Video)
    annotationId = ForeignKeyField(AMTHits)
    sequenceId = CharField()
    socketId = CharField()
    sourceId = CharField()
    destId = CharField()
    created_at = IntegerField(default=int(datetime.now().strftime('%s')))

class Answer(BaseModel):
    id = CharField(primary_key=True, index=True)
    answer = CharField()
    question = ForeignKeyField(Question)
    video = ForeignKeyField(Video)
    annotationId = ForeignKeyField(AMTHits)
    sequenceId = CharField()
    socketId = CharField()
    sourceId = CharField()
    destId = CharField()
    created_at = IntegerField(default=int(datetime.now().strftime('%s')))

def createDatabaseTables():
    database.connect()

    if not Video.table_exists():
        database.create_table(Video)
    #if not Label.table_exists():
    #    database.create_table(Label)
    if not AMTHits.table_exists():
        database.create_table(AMTHits)
    if not Feedback.table_exists():
        database.create_table(Feedback)
    if not Question.table_exists():
        database.create_table(Question)
    if not Answer.table_exists():
        database.create_table(Answer)
    print "All database tables created."

def fillPilotData():

    split = 'train' # TODO
    print 'Loading Label for ' + split + ' data...'

    kineticsPath =  '/media/hudaalamri/450e4e47-0b67-4b43-b098-505e454af9f6/kinetics/kinetics_train/videos/kinetics_'# TODO
    labelPath  = '/media/hudaalamri/450e4e47-0b67-4b43-b098-505e454af9f6/kinetics/kinetics_train/'# TODO

    f1 = open(os.path.join(labelPath, 'kinetics_' + split + '.json'))
    labelData = json.loads(f1.read())
    f1.close()

    viddir='%s_%_05d_%04d.mp4'
    subtype = split

    video_path = kineticsPath + subtype + '/'
    #video_list = glob.glob(video_path + '*.mp4')

    video_count = 0
    video_list = []

    for filename in os.listdir(video_path):
        video_all = glob.glob(video_path+filename + '/*.mp4')
        for video in video_all:
            video_list.append(video)


    query = "SELECT * FROM video WHERE split = '" + split + "'"

    with con:
        cur = con.cursor()
        cur.execute(query)

        c = 0

        for i in range(cur.rowcount):
            row = cur.fetchone()
            if (video_path + row[1]) in vid:
                video_list.remove(video_path + row[1])
                c +=1
        print(cur.rowcount)
        print c

    count = 0
    new_list = []

    for name in video_list:
        new_list.append(name[len(video_path):])
        count = count +1

    print count

    labels = defaultdict(list)
    start_times = defaultdict(list)
    end_times = defaultdict(list)
    subtypes = defaultdict(list)
    urls = defaultdict(list)

    for youtube_id , value in labelData.iteritems():
        lab =value[u'annotations'][u'label']
        time_start = value['annotations']['segment'][0]
        time_end = value['annotations']['segment'][1]
        subtype = value['subset']
        url = value['url']

        #print(type(lab))
        #print(type(time_start))
        #print(type(time_end))
        #print(type(subtype))
        #print(type(url))

        lab = labelData[youtube_id][u'annotations'][u'label']
        labels[youtube_id].append(str(lab))
        start_times[youtube_id].append(time_start)
        end_times[youtube_id].append(time_end)
        subtypes[youtube_id].append(str(subtype))
        urls[youtube_id].append(str(url))

        videoData = []
    #labelData = []

    for vid in new_list:
        #imgid = caption[imname][0]['image_id']

        start = vid.find('/')
        youtube_id = vid[start+1 : start + 12]

        vidName = vid[start+1 :len(vid)]
        lab = labels[youtube_id]
        starTime = start_times[youtube_id]
        endTime = end_times[youtube_id]
        subType = subtypes[youtube_id]
        url = urls[youtube_id]


        videoData.append({'youtube_id':str(youtube_id),'video_name':str(vidName),'video_Type':'kinetics', 'label':labels[youtube_id], 'split':'train','time_start':starTime,'time_end':endTime,'url':str(url),'numHitsFinished':'0'})
        #labelData.append({'label':str(youtube_id), 'video':str(youtube_id)})

    with database.atomic():
        for idx in range(0, len(videoData), 200):

            s = idx
            e = min(idx+200, len(videoData))
            print(idx)
            print(e)

            Video.insert_many(videoData[s:e]).execute()
        
    print "Video table for " + split + " created."
    
def createRedisQueue():
    print "createRedisQueue called."

    import redis
    r = redis.StrictRedis(host='localhost', port=6379, db=0) # TODO

    with con:
        cur = con.cursor()
        cur.execute("SELECT * FROM video WHERE numHitsFinished = 0")
        #cur.execute("SELECT * FROM video WHERE numHitsFinished = 0")
        r.delete('visdial_queue')
        count = 0
        for i in range(cur.rowcount):
            row = cur.fetchone()
            if(count < 5000): # push 5k images into queue
                print row[1], row[4]
                r.rpush('visdial_queue', row[1])
                count += 1

        print count

## ------------------------------------------------------------------------------------------------------
createDatabaseTables()

fillPilotData()

createRedisQueue()